***1. Intro + Data Setup***

**Supply Chain Analysis means analyzing various components of a Supply Chain to understand how to improve the effectiveness of the Supply Chain to create more value for customers. The dataset used here is from a Fashion and Beauty startup. It is based on the supply chain of skincare, haircare and cosmetic products.**

In [63]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"

In [64]:
data = pd.read_csv("/content/supply_chain_data.csv")


**Read Data**

In [65]:
print(data.head())

  Product type   SKU      Price  Availability  Number of products sold  \
0     haircare  SKU0  69.808006            55                      802   
1     skincare  SKU1  14.843523            95                      736   
2     haircare  SKU2  11.319683            34                        8   
3     skincare  SKU3  61.163343            68                       83   
4     skincare  SKU4   4.805496            26                      871   

   Revenue generated Customer demographics  Stock levels  Lead times  \
0        8661.996792            Non-binary            58           7   
1        7460.900065                Female            53          30   
2        9577.749626               Unknown             1          10   
3        7766.836426            Non-binary            23          13   
4        2686.505152            Non-binary             5           3   

   Order quantities  ...  Location Lead time  Production volumes  \
0                96  ...    Mumbai        29          

**Descriptive Statistics**

In [66]:
print(data.shape)

(100, 24)


In [67]:
print(data.describe())


            Price  Availability  Number of products sold  Revenue generated  \
count  100.000000    100.000000               100.000000         100.000000   
mean    49.462461     48.400000               460.990000        5776.048187   
std     31.168193     30.743317               303.780074        2732.841744   
min      1.699976      1.000000                 8.000000        1061.618523   
25%     19.597823     22.750000               184.250000        2812.847151   
50%     51.239831     43.500000               392.500000        6006.352023   
75%     77.198228     75.000000               704.250000        8253.976921   
max     99.171329    100.000000               996.000000        9866.465458   

       Stock levels  Lead times  Order quantities  Shipping times  \
count    100.000000  100.000000        100.000000      100.000000   
mean      47.770000   15.960000         49.220000        5.750000   
std       31.369372    8.785801         26.784429        2.724283   
min        0

**Data Quality Checks**

In [68]:
print(data.isnull().sum())
print()
print(data.duplicated().sum())
print()
print(data.nunique())
print()
print(data.dtypes)

Product type               0
SKU                        0
Price                      0
Availability               0
Number of products sold    0
Revenue generated          0
Customer demographics      0
Stock levels               0
Lead times                 0
Order quantities           0
Shipping times             0
Shipping carriers          0
Shipping costs             0
Supplier name              0
Location                   0
Lead time                  0
Production volumes         0
Manufacturing lead time    0
Manufacturing costs        0
Inspection results         0
Defect rates               0
Transportation modes       0
Routes                     0
Costs                      0
dtype: int64

0

Product type                 3
SKU                        100
Price                      100
Availability                63
Number of products sold     96
Revenue generated          100
Customer demographics        4
Stock levels                65
Lead times                  29
Order qu

***2. KPI Overview & Foundational Metrics***




**KPI Summary Dashboard**

In [69]:
summary_metrics = {
    'Total Revenue': data['Revenue generated'].sum(),
    'Avg Lead Time': data['Lead time'].mean(),
    'Avg Manufacturing Cost': data['Manufacturing costs'].mean(),
    'Avg Defect Rate': data['Defect rates'].mean()
}

In [70]:
import pandas as pd

summary_df = pd.DataFrame.from_dict(summary_metrics, orient='index', columns=['Value'])
summary_df

,Value
Total Revenue,577604.818738
Avg Lead Time,17.080000
Avg Manufacturing Cost,47.266693
Avg Defect Rate,2.277158


In [71]:
avg_lead_time = data.groupby('Product type')['Lead time'].mean().reset_index()
avg_manufacturing_costs = data.groupby('Product type')['Manufacturing costs'].mean().reset_index()
result = pd.merge(avg_lead_time, avg_manufacturing_costs, on='Product type')
result.rename(columns={'Lead time': 'Average Lead Time', 'Manufacturing costs': 'Average Manufacturing Costs'}, inplace=True)
print(result)

  Product type  Average Lead Time  Average Manufacturing Costs
0    cosmetics          13.538462                    43.052740
1     haircare          18.705882                    48.457993
2     skincare          18.000000                    48.993157


**Profit and Profit Margin Columns**

In [72]:
# Calculate profit and profit margin
data['Profit'] = data['Revenue generated'] - data['Manufacturing costs'] - data['Shipping costs']
data['Profit Margin (%)'] = (data['Profit'] / data['Revenue generated']) * 100

**Profit by Product Type**

In [73]:
import plotly.express as px

profit_by_product = data.groupby('Product type')['Profit'].sum().reset_index()

fig = px.bar(profit_by_product,
             x='Product type',
             y='Profit',
             title='Total Profit by Product Type',
             color='Profit',
             color_continuous_scale='greens')
fig.show()

In [74]:
total_profit = data.groupby('Product type')['Profit'].sum()
print(total_profit)

Product type
cosmetics    160244.331084
haircare     172606.955095
skincare     239472.048328
Name: Profit, dtype: float64


**Profit Margin by Product Type**

In [75]:
margin_by_product = data.groupby('Product type')['Profit Margin (%)'].mean().reset_index()

fig = px.bar(margin_by_product,
             x='Product type',
             y='Profit Margin (%)',
             title='Average Profit Margin by Product Type',
             color='Profit Margin (%)',
             color_continuous_scale='blues')
fig.show()

**3. Revenue & Sales Drivers**

**Product type and Price: Analyzing the Supply Chain by looking at the relationship between the price of the products and the revenue generated by them.**

In [76]:
fig = px.scatter(data, x='Price',
                 y='Revenue generated',
                 color='Product type',
                 hover_data=['Number of products sold'],
                 trendline="ols")
fig.show()

**Sales by Product Type**




In [77]:
avg_margin = data.groupby('Product type')['Profit Margin (%)'].mean()
print(avg_margin)

Product type
cosmetics    99.020531
haircare     98.410154
skincare     98.544647
Name: Profit Margin (%), dtype: float64


In [78]:
sales_data = data.groupby('Product type')['Number of products sold'].sum().reset_index()

pie_chart = px.pie(sales_data, values='Number of products sold', names='Product type',
                   title='Sales by Product Type',
                   hover_data=['Number of products sold'],
                   hole=0.5,
                   color_discrete_sequence=px.colors.qualitative.Pastel)

pie_chart.update_traces(textposition='inside', textinfo='percent+label')
pie_chart.show()

*   **The company derives more revenue from skincare products, and the higher the price of skincare products, the more revenue they generate.**
*   **45% of the business comes from skincare products, 29.5% from haircare, and 25.5% from cosmetics. **

**Total Revenue by Shipping Carrier**

In [79]:
total_revenue = data.groupby('Shipping carriers')['Revenue generated'].sum().reset_index()
fig = go.Figure()
fig.add_trace(go.Bar(x=total_revenue['Shipping carriers'],
                     y=total_revenue['Revenue generated']))
fig.update_layout(title='Total Revenue by Shipping Carrier',
                  xaxis_title='Shipping Carrier',
                  yaxis_title='Revenue Generated')
fig.show()

**Product type: The company is using three carriers for transportation, and Carrier B helps the company in generating more revenue. **

***4. SKU-Level Operational Analysis***

**Analyzing SKUs: There’s a column in the dataset as SKUs.SKU stands for Stock Keeping Units. They’re like special codes that help companies keep track of all the different things they have for sale. Imagine you have a large toy store with lots of toys. Each toy is different and has its name and price, but when you want to know how many you have left, you need a way to identify them. So you give each toy a unique code, like a secret number only the store knows. This secret number is called SKU.**

**Revenue generated by SKU**

In [80]:
revenue_chart = px.line(data, x='SKU',
                        y='Revenue generated',
                        title='Revenue Generated by SKU')
revenue_chart.show()

**Stock Levels by SKU: Stock levels refer to the number of products a store or business has in its inventory.**

In [81]:
stock_chart = px.line(data, x='SKU',
                      y='Stock levels',
                      title='Stock Levels by SKU')
stock_chart.show()

**Calculate Inventory Turnover Ratio**

In [82]:
# To avoid division by zero
data['Inventory Turnover Ratio'] = data['Number of products sold'] / (data['Stock levels'].replace(0, 1))
print(data[['SKU', 'Inventory Turnover Ratio']])

      SKU  Inventory Turnover Ratio
0    SKU0                 13.827586
1    SKU1                 13.886792
2    SKU2                  8.000000
3    SKU3                  3.608696
4    SKU4                174.200000
..    ...                       ...
95  SKU95                 44.800000
96  SKU96                  4.835821
97  SKU97                  1.347826
98  SKU98                 17.226415
99  SKU99                 11.400000

[100 rows x 2 columns]


**Visualize Inventory Turnover by SKU**

In [84]:
import plotly.express as px

fig = px.bar(data,
             x='SKU',
             y='Inventory Turnover Ratio',
             color='Product type',
             title='Inventory Turnover by SKU',
             hover_data=['Stock levels', 'Number of products sold'])

fig.show()

**Flag High Stockout Risk SKUs**

In [89]:
# Define thresholds (customize as needed)
low_stock_threshold = 20
high_turnover_threshold = 2.5

# Filter risky SKUs
stockout_risk = data[(data['Stock levels'] < low_stock_threshold) &
                     (data['Inventory Turnover Ratio'] > high_turnover_threshold)]

stockout_risk = stockout_risk[['SKU', 'Product type', 'Stock levels', 'Inventory Turnover Ratio']].sort_values(by='Inventory Turnover Ratio', ascending=False)
stockout_risk

,SKU,Product type,Stock levels,Inventory Turnover Ratio
34,SKU34,skincare,1,602.000000
47,SKU47,skincare,4,227.500000
78,SKU78,haircare,5,189.200000
4,SKU4,skincare,5,174.200000
68,SKU68,haircare,0,163.000000
33,SKU33,cosmetics,4,154.000000
16,SKU16,skincare,2,140.000000
58,SKU58,skincare,10,89.600000
44,SKU44,cosmetics,13,70.692308
9,SKU9,skincare,14,70.000000


**Overstock Risk (Low Turnover + High Stock)**

In [91]:
# Define overstock conditions
high_stock_threshold = 50
low_turnover_threshold = 0.5

# Filter for overstock risk SKUs
overstock_risk = data[
    (data['Stock levels'] > high_stock_threshold) &
    (data['Inventory Turnover Ratio'] < low_turnover_threshold)
][['SKU', 'Product type', 'Stock levels', 'Inventory Turnover Ratio']].sort_values(
    by='Stock levels', ascending=False)

overstock_risk.reset_index(drop=True, inplace=True)
overstock_risk


,SKU,Product type,Stock levels,Inventory Turnover Ratio
0,SKU45,haircare,93,0.258065


**It's just one product that seems to be overstocked.**

In [93]:
import plotly.express as px

fig = px.bar(overstock_risk,
             x='SKU',
             y='Stock levels',
             color='Product type',
             title='Overstocked SKUs (High Stock, Low Turnover)',
             hover_data=['Inventory Turnover Ratio'])

fig.show()

***5. Cost & Logistics***

**Order Quantity by SKU**

In [94]:
order_quantity_chart = px.bar(data, x='SKU',
                              y='Order quantities',
                              title='Order Quantity by SKU')
order_quantity_chart.show()

**Lead Time vs Manufacturing Cost Analysis**

In [95]:
import plotly.express as px

fig = px.scatter(data,
                 x='Lead time',
                 y='Manufacturing costs',
                 color='Product type',
                 hover_data=['SKU'],
                 title='Lead Time vs Manufacturing Cost by Product Type',
                 trendline='ols')
fig.show()

**Defect Rate vs Lead Time Plot**

In [96]:
fig = px.scatter(data, x='Lead time', y='Defect rates', color='Product type',
                 title='Defect Rate vs Lead Time by Product Type', trendline='ols')
fig.show()

*This plot reveals a positive correlation between lead time and defect rate across all product types — suggesting that longer production or delivery times may increase the likelihood of defects, especially in haircare items.*

**Shipping Costs by Carrier**

In [97]:
shipping_cost_chart = px.bar(data, x='Shipping carriers',
                             y='Shipping costs',
                             title='Shipping Costs by Carrier')
shipping_cost_chart.show()

**In one of the above visualizations, we saw that Carrier B helps the company in generating more revenue. Turns out, it is also the most costly Carrier amongst the three carriers.**

**Cost Distribution by Transportation Mode**

In [98]:
transportation_chart = px.pie(data,
                              values='Costs',
                              names='Transportation modes',
                              title='Cost Distribution by Transportation Mode',
                              hole=0.5,
                              color_discrete_sequence=px.colors.qualitative.Pastel)
transportation_chart.show()

**The visualisation shows that the company spends more on Road and Rail modes of transportation for the transportation of Goods.**

***6. Quality Control: Defects***

**Analyzing Defect Rate**

*The defect rate in the supply chain refers to the percentage of products that have something wrong or are found broken after shipping.*

**Average Defect Rates by Product Type**




In [99]:
defect_rates_by_product = data.groupby('Product type')['Defect rates'].mean().reset_index()

fig = px.bar(defect_rates_by_product, x='Product type', y='Defect rates',
             title='Average Defect Rates by Product Type')
fig.show()

*The bar graph shows that the defect rate of haircare products is higher.*

**Defect Rates by Transportation Mode**

In [100]:
pivot_table = pd.pivot_table(data, values='Defect rates',
                             index=['Transportation modes'],
                             aggfunc='mean')

transportation_chart = px.pie(values=pivot_table["Defect rates"],
                              names=pivot_table.index,
                              title='Defect Rates by Transportation Mode',
                              hole=0.5,
                              color_discrete_sequence=px.colors.qualitative.Pastel)
transportation_chart.show()

*Road transportation results in a higher defect rate, and Air transportation has the lowest defect rate.*

***7. Wrap-Up***

**## 🔚 Key Takeaways & Recommendations**

- *Skincare is the largest contributor to revenue; cosmetics yield the highest profit margin per unit sold.*
- *Several SKUs (e.g., SKU34, SKU47) face stockout risks and need prioritized replenishment.*
- *Overstocked items (e.g., SKU12) should be reviewed for markdowns or promotional clearance.*
- *Carrier B generates the most revenue but is also the most expensive — explore carrier optimization or rate renegotiation.*
- *Haircare has the highest defect rate and longest lead times — investigate vendor quality or packaging issues.*